In [1]:
import json
import os
import pandas as pd


def get_json_files():
    json_files = []
    for file in os.listdir("rawdata"):
        if file.endswith(".json"):
            json_files.append(file)
    return json_files


all_files = get_json_files()

In [2]:
data = pd.read_json("rawdata/" + all_files[1])

In [3]:
data.head()

,title,image_source,price,description,product_url
0,Get Well Sunshine Gift Hamper <br/>(Get Well S...,https://www.gifthampersuk.co.uk/image/cache/ca...,£50.00,Hamper ContentsAll Around The Fire Hot Chocola...,https://www.gifthampersuk.co.uk/care-hampers/g...
1,Recovery Treats Gift Hamper <br/>(Get Well Soo...,https://www.gifthampersuk.co.uk/image/cache/ca...,£45.00,Hamper Contents\r\nAlchemy Tea Rest and Recupe...,https://www.gifthampersuk.co.uk/care-hampers/r...
2,To Perk You Up Gift Hamper <br/>(Get Well Soon...,https://www.gifthampersuk.co.uk/image/cache/ca...,£43.00,Hamper Contents\r\nAnna’s Cappuccino Thins (15...,https://www.gifthampersuk.co.uk/care-hampers/a...
3,A Little Bit Of Calm Gift Hamper <br/>(Get Wel...,https://www.gifthampersuk.co.uk/image/cache/ca...,£60.00,Hamper Contents\r\nAlchemy Tea Howick Earl Gre...,https://www.gifthampersuk.co.uk/care-hampers/a...
4,Coffee Connoisseur's Dream Gift Hamper <br/>(G...,https://www.gifthampersuk.co.uk/image/cache/ca...,£50.00,Hamper ContentsGIFT ITEM: Life & Soul Cafetier...,https://www.gifthampersuk.co.uk/care-hampers/c...


In [5]:
from main import get_soup
from individualScraper import individual_product_info, dump_product_info

data = pd.read_json("rawdata/" + all_files[1])

product_data_list_ = []
for index, row in data.iterrows():
    url = row["product_url"]
    soup = get_soup(url)
    product_info = individual_product_info(soup, product_url=url)
    product_data_list_.append(product_info)
    print(f"Scraped {index} of {len(data)}")

csv_file_name = f"rawdata/{all_files[1].split('.')[0]}.csv"
dump_product_info(product_data_list_, csv_file_name)

Scraped 0 of 9
Scraped 1 of 9
Scraped 2 of 9
Scraped 3 of 9
Scraped 4 of 9
Scraped 5 of 9
Scraped 6 of 9
Scraped 7 of 9
Scraped 8 of 9


In [24]:
import requests
from bs4 import BeautifulSoup

def get_soup(url):
    response = requests.get(url)
    html = response.content
    soup = BeautifulSoup(html, 'html.parser')
    return soup

In [25]:
from bs4 import BeautifulSoup, Comment

def clean_html(html):

    # Remove the head tag, scripts, styles, and comments
    if soup.head:
        soup.head.decompose()
    for element in soup.find_all(["script", "style"]) + soup.find_all(string=lambda text: isinstance(text, Comment)):
        element.decompose()

    # Define unwanted attributes including event handlers (e.g., onclick, onerror)
    unwanted_attributes = ["class", "id", "style", "onclick", "onerror", "onload"]

    # Also remove attributes by pattern (e.g., data-*, aria-*)
    for tag in soup.find_all():
        for attribute in list(tag.attrs):
            if attribute.startswith(("data-", "aria-", "on")) or attribute in unwanted_attributes:
                del tag.attrs[attribute]

    # Remove unwanted tags and additional layout-oriented or non-content tags
    unwanted_tags = ['header', 'footer', 'nav', 'aside', 'svg', 'form', 'input', 'textarea', 'button', 'select']
    for tag in unwanted_tags:
        for element in soup.find_all(tag):
            element.decompose()

    # Unwrap certain tags to keep their content but remove the tag itself
    for tag_name in ['span', 'div', 'section']:
        for tag in soup.find_all(tag_name):
            tag.unwrap()

    return soup.prettify()
